In [209]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 12 23:24:54 2016

@author: stuka

"""

import numpy as np
import nltk
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import os
import io
import re
from collections import defaultdict
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import math
import pandas as pd
import sys
import gensim
from string import punctuation
from scipy.cluster.vq import kmeans,vq

#Cleaning the house
os.getcwd()
os.chdir("/home/christian/Documents/ITAM_CDA/Mineria de texto/final_project/mineria-texto-diario-debates/code/")
os.listdir(".")

path_to_raw = "/home/christian/Documents/ITAM_CDA/Mineria de texto/final_project/mineria-texto-diario-debates/data/"


In [ ]:
#Debe venir de fuera
"""
Christian - pon tu path local en path_to_raw
Mete manualmente archivos a una carpeta_christian y corre este codigo
file_names = [f for f in os.listdir(path_to_raw+'carpeta_christian/') if f.endswith('.txt')]
documentos =[io.open(f,'rt') for f in file_names]


"""

file_names = [f for f in os.listdir(path_to_raw+'raw/') if f.endswith('.txt')]

documentos =[io.open(f,'rt',encoding='ISO-8859-1') for f in file_names]


for file_name in file_names:
    sourceEncoding = "iso-8859-1"
    targetEncoding = "utf-8"
    source = io.open(file_name,'rt',encoding='ISO-8859-1')
    target = open(path_to_raw+'encoded/'+file_name, "w")
    target.write(source.read().encode(targetEncoding))
    
documentos =[io.open(path_to_raw+'encoded/'+f,'rt') for f in file_names]

In [ ]:
#Pruebas
raw = documentos[0].read()

documentos[1]

dir(data[0])
data[0].name
#Codigo para probar stopterm
for documento in documentos:
    print(len(documento.read().replace("Honorable Asamblea:","Honorable asamblea:").split("Honorable asamblea:")))

In [ ]:
    
#Este es el buen codigo
"""
documento trae un file abierto
lo consumo a memoria con read() a un solo string
lo parto segun keyterms = Honorable asamblea
"""
for documento in documentos:
    documento_nombre = documento.name.rsplit('/',1)[1].split('.')[0]
    raw = documento.read()
    for i,tematica in enumerate(raw.replace("Honorable Asamblea:","Honorable asamblea:").split("Honorable asamblea:")):
        with io.open(path_to_raw+'tematicas/'+documento_nombre+'_'+str(i+1)+'.txt', mode="w") as newfile:
            tematica_limpia = mataAcentos(strip_punctuation(tematica.replace('\n',' ')))
            newfile.write(tematica_limpia)

def mataAcentos(s):
    charstosub = pd.DataFrame(zip([u'á', u'é', u'í', u'ó', u'ú',u'"',u'“',u'”',u',',u'\.',u'ñ',u'\!',u'\¡'],[u'a', u'e', u'i', u'o', u'u',u'',u'',u'',u'',u'',u'n',u'',u''])) 
    data = s.lower()
    for row in charstosub.iterrows():
        data = re.sub(row[1][0],row[1][1],data)
    return data
##Este no funciona aun
"""
Remover acentos y puntuacion parece no ser lo mejor


def cleanText(corpus):
    #punctuation = """.,?!:;(){}[]"""
    punctuation = punctuation
    corpus = [z.lower().replace('\n','') for z in corpus]
    corpus = [z.replace('<br />', ' ') for z in corpus]

    #treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s '%c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus
"""

#Esto si funciona
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [142]:
"""
Christian!!!!!
Crea una carpeta prueba y mete alli documentos de prueba para RAKE
"""
file_names_tematicas = [f for f in os.listdir(path_to_raw+'tematicas/') if f.endswith('.txt')]
sentences = []
i=0
dicto = {}
for doc in file_names_tematicas:
    f = io.open(path_to_raw+'tematicas/'+doc,'rt')
    sentences.append(gensim.models.doc2vec.LabeledSentence(f.read().split(),[doc]))
    f.close()
    
    
#Pruebas
type(sentences[0])
#sentences[0:5]
#print(sentences[0:5])


gensim.models.doc2vec.LabeledSentence

In [143]:
#Modelo
min_count = 5 # Ignora todo documento que tenga cinco palabras o menos
size = 60 # Numero de dimensiones (tanto de las palabras como de los documentos)
window = 10 # Cuantas palabras hacia adelante / atrás usa para trabajar
 
model = gensim.models.doc2vec.Doc2Vec(sentences,size = size, window = window, min_count = min_count)

In [144]:
dir(model)
len(model.vocab)
model.similarity('pri','pan')

0.89159278462155822

In [145]:
def checa(word):
    for i in model.most_similar(word):
        print(i)
        
checa(u'fraudes')

(u'procesos', 0.8570495843887329)
(u'deseos', 0.7876545190811157)
(u'padrones', 0.7757041454315186)
(u'aumentos', 0.768700122833252)
(u'argumentos', 0.7674710750579834)
(u'comprobantes', 0.7639429569244385)
(u'nombres', 0.763940691947937)
(u'escrutinios', 0.7610893249511719)
(u'triunfos', 0.7582401037216187)
(u'razonamientos', 0.7577353119850159)


In [147]:
#print model.docvecs.most_similar(["SENT_1"])
#model.docvecs['SENT_37']
#dir(model.docvecs)
#model.docvecs.doctags
#dir(model)

### K means

In [148]:
mylist = []
tags=[]
for doc in model.docvecs.doctags:
    tags.append(doc)    
    mylist.append(model.docvecs[doc])



mat = np.array(mylist)

In [149]:
for i in range(10):
    agrupamiento = KMeans(n_clusters = i+1)
    agrupamiento.fit(mat)
#    mylist['clasificacion_'] = agrupamiento.labels_.tolist()

In [150]:
from scipy.cluster.vq import kmeans,vq
K = range(1,10)

KM = [kmeans(mat,k) for k in K]
centroids = [cent for (cent,var) in KM] 
avgWithinSS = [var for (cent,var) in KM] # suma de cuadrados promedio intra-cluster 

In [151]:
##### plot ###
kIdx = 4

# elbow curve
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, avgWithinSS, 'b*-')
ax.plot(K[kIdx], avgWithinSS[kIdx], marker='o', markersize=12, 
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Numero de clusters')
plt.ylabel('Suma de cuadrados promedio intra-cluster')
plt.title('Criterio del codo para K-Medias')

plt.show()

In [152]:
# KMeans con el resultado del criterio del codo
agrupamiento = KMeans(n_clusters = 5)
agrupamiento.fit(mat)
agrupamiento.labels_

array([3, 0, 3, ..., 0, 0, 4], dtype=int32)

In [189]:
agrupados = pd.DataFrame(zip(tags, agrupamiento.labels_.tolist()))
n_clusters=5

In [192]:
lista_cluster = [[],[],[],[],[]]
lista_cluster

[[], [], [], [], []]

In [197]:
for i in range(len(agrupados)):
    lista_cluster[agrupados[1][i]].append(agrupados[0][i])

lista_cluster[0]

['dd_1970_328.txt',
 'dd_1964_142.txt',
 'dd_1970_202.txt',
 'dd_1976_237.txt',
 'dd_1970_320.txt',
 'dd_1976_391.txt',
 'dd_1964_149.txt',
 'dd_1958_133.txt',
 'dd_1970_311.txt',
 'dd_1976_408.txt',
 'dd_1952_99.txt',
 'dd_1976_323.txt',
 'dd_1964_260.txt',
 'dd_1976_291.txt',
 'dd_1964_263.txt',
 'dd_1964_221.txt',
 'dd_1970_281.txt',
 'dd_1946_10.txt',
 'dd_1976_260.txt',
 'dd_1970_208.txt',
 'dd_1958_140.txt',
 'dd_1970_254.txt',
 'dd_1952_102.txt',
 'dd_1952_141.txt',
 'dd_1970_245.txt',
 'dd_1970_238.txt',
 'dd_1964_272.txt',
 'dd_1970_239.txt',
 'dd_1946_89.txt',
 'dd_1976_417.txt',
 'dd_1952_106.txt',
 'dd_1946_102.txt',
 'dd_1970_154.txt',
 'dd_1946_31.txt',
 'dd_1964_222.txt',
 'dd_1964_144.txt',
 'dd_1958_55.txt',
 'dd_1946_19.txt',
 'dd_1976_422.txt',
 'dd_1976_272.txt',
 'dd_1946_150.txt',
 'dd_1946_133.txt',
 'dd_1964_238.txt',
 'dd_1946_42.txt',
 'dd_1952_120.txt',
 'dd_1958_144.txt',
 'dd_1964_193.txt',
 'dd_1964_127.txt',
 'dd_1964_115.txt',
 'dd_1964_172.txt',
 'dd_19

### RAKE

In [211]:
import rake
import operator

In [216]:
! pwd

/home/christian/Documents/ITAM_CDA/Mineria de texto/final_project/mineria-texto-diario-debates/code


In [225]:
rake_object = rake.Rake("stop_words_spanish.txt", 5, 5, 100)


In [226]:
sample_file = open(path_to_raw+'clusters_to_rake/'+"cluster_0.txt", 'r')
#sample_file = open(path_to_raw+'tematicas_to_rake/'+"dd_1952_12.txt", 'r')
text = sample_file.read()
keywords = rake_object.run(text)
print "Keywords:", keywords

Keywords: [('son leg\xc3\xadtimas y v\xc3\xa1lidas', 26.652636157192433), ('c\xc3\xa1mara de diputados del h', 26.117857273152683), ('primera comisi\xc3\xb3n revisora de credenciales', 24.44771269298402), ('son diputados de mayor\xc3\xada a', 24.23788446445316), ('quincuag\xc3\xa9sima legislatura del h', 22.53728066122021), ('votaci\xc3\xb3n econ\xc3\xb3mica se pregunta', 22.0475650420007), ('comisi\xc3\xb3n revisora de credenciales', 20.54045462846789), ('constituci\xc3\xb3n pol\xc3\xadtica de', 20.169754285297593), ('julio del presente a\xc3\xb1o', 19.998009888616068), ('son diputados al congreso de', 18.568596403112533), ('c\xc3\xa1mara de diputados', 16.986925590551657), ('son diputados propietario y suplente', 16.777962342724194), ('elecciones a diputados federales celebradas', 16.152053604870094), ('partido aut\xc3\xa9ntico de', 15.530283916630449), ('partido acci\xc3\xb3n nacional', 15.497404096366418), ('comisi\xc3\xb3n somete a', 15.365036159187616), ('primera comisi\xc3\xb3n 

In [ ]:
io "Keywords:", keywords

In [236]:
for item,ki in keywords:
    print item,ki

son legítimas y válidas 26.6526361572
cámara de diputados del h 26.1178572732
primera comisión revisora de credenciales 24.447712693
son diputados de mayoría a 24.2378844645
quincuagésima legislatura del h 22.5372806612
votación económica se pregunta 22.047565042
comisión revisora de credenciales 20.5404546285
constitución política de 20.1697542853
julio del presente año 19.9980098886
son diputados al congreso de 18.5685964031
cámara de diputados 16.9869255906
son diputados propietario y suplente 16.7779623427
elecciones a diputados federales celebradas 16.1520536049
partido auténtico de 15.5302839166
partido acción nacional 15.4974040964
comisión somete a 15.3650361592
primera comisión dictaminadora 15.1869684723
económica se pregunta 15.1633130649
fracciones i y iii 15.1051247398
votación económica 15.0843777097
está a discusión 15.0067890997
orden del día 14.9515318902
de nuevo león 14.9256564643
sala de comisiones de 14.8176102129
diputados al congreso de 14.5759873861
legislatura 

In [254]:
oraciones_clave_0 = []
for item,ki in keywords:
    oraciones_clave_0.append(item)
len(oraciones_clave_0)

176

In [248]:
with io.open(path_to_raw+'resultados_rake/'+'cluster_0_keywords'+'.txt', mode="w") as newfile:
    for i,j in keywords:
            line = str(i)+" - "+str(j)
            newfile.writelines(line.encode("utf-8"))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 7: ordinal not in range(128)

In [249]:
sample_file1 = open(path_to_raw+'clusters_to_rake/'+"cluster_1.txt", 'r')
text1 = sample_file1.read()
keywords1 = rake_object.run(text1)
print "Keywords:", keywords1

Keywords: [('hacienda y cr\xc3\xa9dito p\xc3\xbablico', 26.96456292062732), ('c\xc3\xa1mara de diputados del h', 23.790774278766172), ('secretar\xc3\xada de hacienda y', 19.869820847776968), ('constituci\xc3\xb3n pol\xc3\xadtica de', 19.69126606809537), ('secretar\xc3\xada de gobernaci\xc3\xb3n', 19.52024994817861), ('sesi\xc3\xb3n y se cita', 19.040634968821543), ('secretar\xc3\xada de hacienda', 15.9702083928792), ('orden del d\xc3\xada', 15.887771907754393), ('votaci\xc3\xb3n econ\xc3\xb3mica', 15.62382315923373), ('que se refiere a', 15.310921380321435), ('c\xc3\xa1mara de diputados', 15.008434078550248), ('secretario altamirano mar\xc3\xadn', 15.00382426452266), ('c\xc3\xa1mara de senadores', 14.982508445685227), ('est\xc3\xa1 a discusi\xc3\xb3n', 14.938170945756815), ('secretario guerrero esquivel fernando', 14.830846210444783), ('se procede a recoger', 14.683731982828292), ('departamento del distrito federal', 14.654041157859226), ('banco de m\xc3\xa9xico', 14.608144578512546), 

In [255]:
for item,ki in keywords1:
    print item,ki
len(keywords1)

hacienda y crédito público 26.9645629206
cámara de diputados del h 23.7907742788
secretaría de hacienda y 19.8698208478
constitución política de 19.6912660681
secretaría de gobernación 19.5202499482
sesión y se cita 19.0406349688
secretaría de hacienda 15.9702083929
orden del día 15.8877719078
votación económica 15.6238231592
que se refiere a 15.3109213803
cámara de diputados 15.0084340786
secretario altamirano marín 15.0038242645
cámara de senadores 14.9825084457
está a discusión 14.9381709458
secretario guerrero esquivel fernando 14.8308462104
se procede a recoger 14.6837319828
departamento del distrito federal 14.6540411579
banco de méxico 14.6081445785
a proceder a recoger 14.5827310169
ciudadano diputado de votar 14.2382829769
sala de comisiones de 14.1863456724
presupuesto de egresos de 13.9159151206
constitución general de 13.9123041112
ciudad de méxico 13.7307745622
señor presidente de 13.4478585474
secretarías de 11.8222681885
párrafo anterior 11.6570298125
secretaría de 11.56

414

In [242]:
[item[0] for item in keywords]

['son leg\xc3\xadtimas y v\xc3\xa1lidas',
 'c\xc3\xa1mara de diputados del h',
 'primera comisi\xc3\xb3n revisora de credenciales',
 'son diputados de mayor\xc3\xada a',
 'quincuag\xc3\xa9sima legislatura del h',
 'votaci\xc3\xb3n econ\xc3\xb3mica se pregunta',
 'comisi\xc3\xb3n revisora de credenciales',
 'constituci\xc3\xb3n pol\xc3\xadtica de',
 'julio del presente a\xc3\xb1o',
 'son diputados al congreso de',
 'c\xc3\xa1mara de diputados',
 'son diputados propietario y suplente',
 'elecciones a diputados federales celebradas',
 'partido aut\xc3\xa9ntico de',
 'partido acci\xc3\xb3n nacional',
 'comisi\xc3\xb3n somete a',
 'primera comisi\xc3\xb3n dictaminadora',
 'econ\xc3\xb3mica se pregunta',
 'fracciones i y iii',
 'votaci\xc3\xb3n econ\xc3\xb3mica',
 'est\xc3\xa1 a discusi\xc3\xb3n',
 'orden del d\xc3\xada',
 'de nuevo le\xc3\xb3n',
 'sala de comisiones de',
 'diputados al congreso de',
 'legislatura del congreso de',
 'conclusi\xc3\xb3n de que',
 'comisi\xc3\xb3n federal elect

In [158]:
#nltk.download()
nltk.corpus.stopwords.words('spanish')

[u'de',
 u'la',
 u'que',
 u'el',
 u'en',
 u'y',
 u'a',
 u'los',
 u'del',
 u'se',
 u'las',
 u'por',
 u'un',
 u'para',
 u'con',
 u'no',
 u'una',
 u'su',
 u'al',
 u'lo',
 u'como',
 u'm\xe1s',
 u'pero',
 u'sus',
 u'le',
 u'ya',
 u'o',
 u'este',
 u's\xed',
 u'porque',
 u'esta',
 u'entre',
 u'cuando',
 u'muy',
 u'sin',
 u'sobre',
 u'tambi\xe9n',
 u'me',
 u'hasta',
 u'hay',
 u'donde',
 u'quien',
 u'desde',
 u'todo',
 u'nos',
 u'durante',
 u'todos',
 u'uno',
 u'les',
 u'ni',
 u'contra',
 u'otros',
 u'ese',
 u'eso',
 u'ante',
 u'ellos',
 u'e',
 u'esto',
 u'm\xed',
 u'antes',
 u'algunos',
 u'qu\xe9',
 u'unos',
 u'yo',
 u'otro',
 u'otras',
 u'otra',
 u'\xe9l',
 u'tanto',
 u'esa',
 u'estos',
 u'mucho',
 u'quienes',
 u'nada',
 u'muchos',
 u'cual',
 u'poco',
 u'ella',
 u'estar',
 u'estas',
 u'algunas',
 u'algo',
 u'nosotros',
 u'mi',
 u'mis',
 u't\xfa',
 u'te',
 u'ti',
 u'tu',
 u'tus',
 u'ellas',
 u'nosotras',
 u'vosostros',
 u'vosostras',
 u'os',
 u'm\xedo',
 u'm\xeda',
 u'm\xedos',
 u'm\xedas',
 u

In [256]:
sample_file2 = open(path_to_raw+'clusters_to_rake/'+"cluster_2.txt", 'r')
#sample_file = open(path_to_raw+'tematicas_to_rake/'+"dd_1952_12.txt", 'r')
text2 = sample_file2.read()
keywords2 = rake_object.run(text2)
print "Keywords:", keywords2

Keywords: [('c\xc3\xa1mara de diputados del h', 23.135610888675192), ('votaci\xc3\xb3n econ\xc3\xb3mica se pregunta', 22.909033135962733), ('se\xc3\xb1oras y se\xc3\xb1ores', 19.16956425095261), ('constituci\xc3\xb3n pol\xc3\xadtica de', 18.79631394458803), ('secretar\xc3\xada de gobernaci\xc3\xb3n', 18.58045937870785), ('a trav\xc3\xa9s de', 16.048985114622084), ('votaci\xc3\xb3n econ\xc3\xb3mica', 15.940887560703798), ('reyes jos\xc3\xa9 mar\xc3\xada', 15.628892396439166), ('partido aut\xc3\xa9ntico de', 15.333312230391124), ('orden del d\xc3\xada', 15.30482943876068), ('partido acci\xc3\xb3n nacional', 14.94064345834785), ('est\xc3\xa1 a discusi\xc3\xb3n', 14.564829683096068), ('c\xc3\xa1mara de diputados', 14.486617335108031), ('pueblo de m\xc3\xa9xico', 14.3566014271252), ('c\xc3\xa1mara de senadores', 14.263742717673187), ('primera comisi\xc3\xb3n dictaminadora', 14.256214861420148), ('ciudad de m\xc3\xa9xico', 14.214800622887093), ('sala de comisiones de', 14.052804479755233), (